# CS 506 Spring 2021 - HW3
## Social Networks and Recommendataion Systems
#### Total: 25 Points

##### Package Limitations: None

### Name: Camden Kronhaus
### BU ID: U79620042
### BU Email: kronhaus@bu.edu
#### People you worked with on this hw: None

### 1. Background

In this homework, you will try to recommend new collaborations to researchers of the Machine Learning community. Our approach will follow the guidelines of collaborative filtering: “**If your past behavior/preferences were similar to some other user’s, your future behavior may be as well**”. As an example, imagine you like Rolling Stones, Beatles and Jimmy Hendrix. It turns out that most people that like the aforementioned artists, are also fans of Eric Clapton. Then, it is very likely that if you listen to Eric Clapton’s music, you will like it as well.

In this assignment you will implement a **collaborative filtering recommendation system** for suggesting new collaborations to Machine Learning researchers.

**A network as a graph**: A graph or network represents relationships among different entities (users of a social network, researchers, products, etc.). Those entities are represented as nodes and the relationships between them (friends on Facebook, co-authors of a research paper, products purchased together) as edges. When there is an edge between two nodes, x and y, we say that y is a neighbor (or friend) of x (and also - as the graphs we consider are undirected - x is also a neighbor of y).

**Representing a graph in Python**: A widely used library in Python, for representing graphs is [NetworkX](https://networkx.github.io/documentation/stable/). You can read the documentation for more information on how to use this library.

### 2. Recommend new collaborations - The ML Community case 

In order to provide new collaborations and test the efficiency of the methods used, you are given two files (you can find them on piazza):

- ”old edges.txt”: In this file, every line contains the names of two re- searchers that have co-authored a paper in one of the top Machine Learn- ing conferences (NeurIPS, ICLR, ICML) between 2010 and 2016.
- ”new edges.txt”: In this file, every line contains the names of two re- searchers (from those existing in the above file) that formed a new (non- existing before) collaboration, in either 2017 and 2018.

With the first file in hand, you will answer the following question:
“For author X, list some non-collaborators in order, starting with the best col- laborator recommendation and ending with the worst”. A non-friend is a user who is not X and is not a collaborator of X. Depending on the recommendation algorithm you are going to choose, the list may include all non-collaborators or some of them.

Then, using the second file, with actual new collaborations formed in the next 3 years, you will test the efficiency of these algorithms.


### Tasks
a) [3 pts.] Write a function that reads the file “old edges.txt” and create a graph using NetworkX. (This is a tab-separated value (TSV) file, you may use packages such as Pandas to read it. )


In [1]:
# You can add functions, inputs outputs to existing functions. 
# Please do NOT change name of the existing functions

from typing import Tuple, List, Dict, Callable
import networkx as nx

def read_old_edges(file_path: str) -> nx.Graph:
    """
    read old edges text file and return a NetworkX graph
    
    :param file_path: string file path to old_edges.txt
    :return: network graph instance of the graph
    """
    
    return nx.read_edgelist(file_path, delimiter='\t', nodetype=str)

b) [3 pts.] Write a function that reads the file “new edges.txt” and for each author, keeps track of the new collaborations this user formed during 2017-2018.


In [2]:
def read_new_edges(file_path: str):
    """
    read new edges text file and for each author, keeps track of the new collaborations this user formed during.
    
    :param file_path: string file path to new_edges.txt
    :return: network graph of new edges between nodes
    """


    return nx.read_edgelist(file_path, delimiter='\t', nodetype=str)

In [ ]:
# G = read_old_edges('./old_edges.txt')
# G = read_new_edges('./new_edges.txt')

In 2017 and 2018, there were 1,757 new edges formed between existing authors. For the next tasks, pick (and recommend new collaborations for) those authors that formed at least 10 new connections between 2017-2018. In the remaining, when we talk about author X, we refer to one of those authors.

c) [5 pts.] **Recommend by number of common friends**

The intuition behind this recommendation algorithm is that if non-friend Y is your friend’s friend, then maybe Y should be your friend too. If person Y is the friend of many of your friends, then Y is an even better recommendation. 

In [3]:
def common_friends_number(graph: nx.Graph, author) -> List[str]:
    """
    Return list of authors who have a common neighbor as 
    given author sorted by number of common friends. 
    
    :param graph: collaboration graph in nextworkX
    :return: list of new collaborators' name to recommend
    """
    recs = {}
    friends = dict.fromkeys(list(graph.neighbors(author)), 1) # Friends of author, make dictionary for faster lookup
    friends[author] = 1 # Add author to list of friends to prevent themself from being added
    for friend in friends:
        mutuals = list(graph.neighbors(friend))
        for mutual in mutuals:
            if (not mutual in friends): # If a mutual is not already a friend, add it, else ignore
                # If alreaday in dict, increase value, otherwise add as new rec with 1
                if(recs.get(mutual) == None):
                    recs[mutual] = 1
                else:
                    recs[mutual] += 1
    return sorted(recs, key=recs.get, reverse=True)
    

In [7]:
common_friends_number(graph=read_old_edges('./old_edges.txt'), author="Adarsh Prasad")

['Holger Hoos',
 'Kevin Leyton-Brown',
 'Alexey Dosovitskiy',
 'Joschka Boedecker',
 'Martin A. Riedmiller',
 'Thomas Brox',
 'Manuel Watter']

d) [5 pts.] **Make recommendations using Jaccard’s Index**

If Γ(X) is the set of neighbors of X, then the metric we used in part (c), assigns to a non-friend y, the following recommendation score (with respect to X): score(y) = |Γ(X)∩Γ(y)|. Jaccard’s Index scales this score by taking into account the union of X and Y ’s neighbors. Intuitively, X and Y are more similar, if what they have in common is as close as possible to what they have together.


In [4]:
def jaccard_index(graph: nx.Graph, author) -> List[str]:
    """
    Return list of authors who have a common neighbor as 
    given author sorted by Jaccard Index (see pdf for equation) 
    
    :param graph: collaboration graph in nextworkX
    :return: list of new collaborators' name to recommend
    """
    recs = {}
    num_friends = len(list(graph.neighbors(author)))
    friends = dict.fromkeys(list(graph.neighbors(author)), 1) # Friends of author, make dictionary for faster lookup
    friends[author] = 1 # Add author to list of friends to prevent themself from being added
    for friend in friends:
        mutuals = list(graph.neighbors(friend))
        for mutual in mutuals:
            if (not mutual in friends): # If a mutual is not already a friend, add it, else ignore
                # If alreaday in dict, increase value, otherwise add as new rec with 1
                if(recs.get(mutual) == None):
                    recs[mutual] = 1
                else:
                    recs[mutual] += 1
    # Divide each score gathered by the total number of friends of X and y, minus the mutuals
    for rec in recs:
        score = recs.get(rec)
        score /= (num_friends + len(list(graph.neighbors(rec))) - recs.get(rec))
        recs[rec] = score

    
    # Ties are arbitrarily broken by first 10 of Timsort
    return sorted(recs, key=recs.get, reverse=True)
    

In [ ]:
jaccard_index(graph=read_old_edges('./old_edges.txt'), author="Andrew C. Miller")

e)  [5 pts.] **Make recommendations using Adamic/Adar Index**

For part (c), we made recommendations using common neighbors. However, when assigning a score to Y , instead of just taking a count of the number of common neighbors, we take a weighted sum of them, where the weight of each common neighbor of X and Y , call her Z, is the inverse of the logarithm of the number of Z’s neighbors. In that way, we value more common neighbors that are more selective.

In [5]:
from math import log
def adamic_adar_index(graph: nx.Graph, author) -> List[str]:
    """
    Return list of recommendations of a given author sorted 
    by Adamic / Adar Index (see pdf for equation) 
    
    :param graph: collaboration graph in nextworkX
    :return: list of new collaborators' name to recommend
    """
    recs = {}
    num_friends = len(list(graph.neighbors(author)))
    friends = dict.fromkeys(list(graph.neighbors(author)), 1) # Friends of author, make dictionary for faster lookup
    friends[author] = 1 # Add author to list of friends to prevent themself from being added
    for friend in friends:
        mutuals = list(graph.neighbors(friend))
        for mutual in mutuals:
            if (not mutual in friends): # If a mutual is not already a friend, add it, else ignore
                if(recs.get(mutual) == None):
                    recs[mutual] = 1 / log(len(mutuals))
                else:
                    recs[mutual] += 1 / log(len(mutuals))

    return sorted(recs, key=recs.get, reverse=True)
    

In [ ]:
adamic_adar_index(graph=read_old_edges('./old_edges.txt'), author="Andrew C. Miller")

f) [4 pts.] **How good are the recommendations we make?** 

Previously, you implemented 3 functions, that given a user X provide recommendations for this user. In this task, you will check how good these recommendations are using the actual new connections formed during 2017-2018.

You will use two different ways, to calculate the efficiency of every approach:

- For each user X, take the 10 first recommendations for this user, and calculate the number of them that were actually formed during 2017-2018. You should report the average among users X.

- For each newly formed collaboration of user X, calculate the rank of this collaboration (the index where this new node Y appears in the recommendations list for X). Report the average among newly formed edges.

In [6]:
def list_popular() -> List:
    '''
    Returns a list of authors who made at least 10 new connections (have 10 edges in new_edges.txt)
    '''
    G = read_new_edges('./new_edges.txt')
    authors = []
    for author in list(G.nodes()):
        if len(G.edges(author)) >= 10:
            authors.append(author)
    

    return authors

In [7]:
populars = list_popular()

In [22]:
def top_k_recommendation_accuracy(graph: nx.Graph, index_method: Callable, new_graph: nx.Graph) -> float:
    """
    Among top k recommendations of every user, return average number 
    of recommendations that becomes reality
    
    :param graph: old_edges graph in nextworkX
    :param index_method: function that is used to make recommendation
    :param new_graph: new_edges graph in nextworkX
    :return: average accuracy of predictions among all users
    """
    k = 10
    authors = list_popular()
    avg = 0 # Average accuracy across all authors
    # Calculate a accuracy for each author and maintain an average
    for author in authors:
        correct = 0 # Keeps track of how many of the top 10 recs are correct
        recs = index_method(graph=graph, author=author)[0:k] # Top 10 recs
        true = list(new_graph.neighbors(author)) # New friends actually formed
        # If a recommended author is in the true neighbors formed, increment correct
        for rec in recs:
            if (rec in true):
                correct += 1
        avg += correct / k


    return avg / len(authors)
    
    
def new_collaboration_rank(graph: nx.Graph, index_method: Callable, new_graph: nx.Graph) -> float:
    """
    Among the new collaborations of every user, return average rank 
    of the collaboration calculated by the function

    If not in the list of reccomendations, rank is set to the next "open" rank
    
    :param graph: collaboration graph in nextworkX
    :param index_method: function that is used to make recommendation
    :param new_graph: new_edges graph in nextworkX
    :return: average rank of predictions among all users
    """
    avg_total = 0 # Average rank among all authors
    authors = list_popular()
    for author in authors:
        recs = index_method(graph=graph, author=author)
        true = list(new_graph.neighbors(author)) # New friends actually formed
        avg_rank = 0 # Average rank for a given author
        rank_adder = len(recs) # Allows "appending" of rank        
        for friend in true:
            if (friend in recs):
                rank = recs.index(friend)
            else:
                rank = rank_adder
            rank_adder += 1
            avg_rank += rank
        avg_rank /= len(true)
        avg_total += avg_rank


    return avg_total / len(authors)

In [18]:
print(top_k_recommendation_accuracy(graph = read_old_edges('./old_edges.txt'), index_method = common_friends_number, new_graph=read_new_edges('./new_edges.txt')))
print(top_k_recommendation_accuracy(graph = read_old_edges('./old_edges.txt'), index_method = jaccard_index, new_graph=read_new_edges('./new_edges.txt')))
print(top_k_recommendation_accuracy(graph = read_old_edges('./old_edges.txt'), index_method = adamic_adar_index, new_graph=read_new_edges('./new_edges.txt')))


0.11463414634146338
0.08048780487804878
0.10000000000000002


In [24]:
print(new_collaboration_rank(graph = read_old_edges('./old_edges.txt'), index_method = common_friends_number, new_graph=read_new_edges('./new_edges.txt')))
print(new_collaboration_rank(graph = read_old_edges('./old_edges.txt'), index_method = jaccard_index, new_graph=read_new_edges('./new_edges.txt')))
print(new_collaboration_rank(graph = read_old_edges('./old_edges.txt'), index_method = adamic_adar_index, new_graph=read_new_edges('./new_edges.txt')))

83.08663740338268
87.71145227830893
85.38917878647678


e) [**Bonus Question**] [2 pts.]
Doing some literature search, suggest your own algorithm for recommend- ing new links to a user X. Argue about the choice you make, why it makes sense to suggest users that way? How is the efficiency of this algorithm, compared to the ones you implemented in parts (c), (d) and (e)?

The complexity of each of the proposed algorithms is something in the neighborhood of O(N^3) if N = # number of nodes, given that for each X, you have to iterate through each friend of X's friends and friend of each mutual friend of X.

For a better algorithm, one might include a score that involves duplicate edges that are filtered out in the above algorithms. If author X and friend Z collaborated on multiple papers together, then mutual friends y may have a higher liklehood of working with X. This wouldn't introduce much complexity at all and could be used in conjuction with the scores presented in the previous algorithms.

Another aspect that could be involved is mutual friends of mutual friends. If author X has mutual friend y through Z, and friend y is also friends with the friend of another Z, then that can be value added to the score. This could be introduced with however many levels deep the developer wants to go, although introducing exponential complexity with each added level. Most likely this should also be added to the score in a inverted relation with the amount of levels deeper one goes such as. After some literature resarrch, this seems to be along the lines of the P-rank formula which is off the same complexity of SimRank which appearrs to be more or less the formula used in part c. 

A third aspect that could be invovled is adding directionality to the graph. By making the graph directed, we could diffrentiate between X working with Z and Z working with X. The difference would lie, if the data provided were provided as such, by who "initiates" the collaboration. If X "recruits" Z to work with them, they are more likely to be the ones to collaborate with friends of Z, y, then friends of another Z who chose to be the ones to collaborate with X. his could simply be incorporated by adding a multipler > 1 to the score added from X working with Z vs Z working with X. This aspect of the score would be contigent on the data being provided as such, so it may not be possible to incorporate depending on how the data is represented. This wouldn't technically add any efficieny problems, since at worst it doubles the size of N from using an undirected graph.

We could also scale the score of each non-friend y by the number of friends they have, somewhat utilizing intuition behind the Google's PageRank algorithm. For a given non-friend y, there score may be scaled by the number of friends they have. Intuitively, if a non-friend y has many friends, they may be considered more important and therefore more likely to eventually work with author X. Again, this does not add much complexity or inefficiencies to the already existing metrics being used in previous problems.